# Question 2 (BLP)

We assume the demand model $$u_{ijt} = X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{i}p_{jt} + \xi_{jt} + \epsilon_{ijt}$$ and use BLP to estimate it. The steps are detailed below:

## Step 1: Simulate draws

The logit specification of our model implies that our shares can be written as an integral of fractions of utility components. Specifically, we can adapt the formulas from class to our model: $$s_{jt} = \int \frac{X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{it}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{i} + \alpha p_{mt} + \sigma_{I}I_{it}p_{mt} + \xi_{mt}}dP_{I}(I)dP_{\nu}(\nu)$$ 
While this integral cannot be solved analytically, we can simulate random draws of income (uniformly over the income data we have for each product-market) and normal demand shocks to estimate shares conditional on demand parameters:
$$\hat{s}_{jt} = \sum_{r}\frac{X_{jt}\beta + \sigma_{B}\nu_{r} + \alpha p_{jt} + \sigma_{I}I_{rt}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{r} + \alpha p_{mt} + \sigma_{I}I_{rt}p_{mt} + \xi_{mt}}$$
We code a function that returns this share while taking demand parameters as input below.

To get closer to the notation used in the lecture notes ($\beta$ is used differently in the problem set,) we will express the deterministic component of utility as $\delta_{jt}$ = $X_{jt}\beta + \alpha p_{jt} + \xi_{jt}$ and the variables that interact with the idiosyncratic components as $\sigma = (\sigma_{B},\sigma_{I})$.

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy

data = pd.read_csv('./cleaned_data/data.csv')
data = data.drop(columns='Unnamed: 0')

demo = pd.read_csv('./PS1_Data/OTCDemographics.csv',sep='\t')
data = pd.merge(data,demo,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
data = data[['store','week','brand','sales_','price_','prom_', 'brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','count', 'ms_naught', 'ms_by_store_week']]

# data['branded'] = 1 - any(data[[f'brand_{i}' for i in range(10,12)]])
data.head()

instr = pd.read_csv('./PS1_Data/OTCDataInstruments.csv',sep='\t')
instr = instr.drop(columns=['store','week','brand','avoutprice'])

In [ ]:
import random
R = 100

# Simulate draws R=100 times and average out shares
def sim_shares(master,sigma_B,sigma_I):

    # Keep covariates and intermediate variables
    master = master[['store','week','brand','ms_by_store_week','sales_','price_','prom_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','w_old','w_new','count']]
    
    # tot is total of all simulated shares
    # master['tot'] = 0
    total = pd.Series([0 for _ in range(len(master))])
    
    nus = np.random.standard_normal(R)
    # Simulate draws
    for i in range(R):
        data_copy = master.copy()
        # Demand shock
        nu = nus[i]
        # Choose income randomly
        hh = random.randint(1,20)

        data_copy['V'] = data_copy['w_old']*np.exp(sigma_B*nu + sigma_I*data_copy[f'hhincome{hh}']*data_copy['price_'])
        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        s = data_copy['V']/data_copy['sum']
        total += s

    # Average share
    master['est'] = total/R
    return master

## Step 2: Contraction Mapping

Given $sigma_{B}, sigma_{I}$, we can iterate the contraction mapping in the lecture notes to approximate a value of $\delta_{jt}$ for each product-market pair that results in a share close to the actual shares in the data. The equation we iterate to approximate $\delta_{jt}$ is:
$$\exp(\delta^{i+1}_{jt}) = \exp(\delta^{i}_{jt})\frac{s_{jt}^{0}}{s_{jt}(\delta^{i}_{jt},\beta)}$$

In [ ]:
# Calculate delta: deterministic component of jt-level utility
def calc_delta(orig,sigma_B,sigma_I,delta=None):
    # Initialize search values and threshold
    epsilon = 0.01
    orig['w_old'] = np.exp(delta) if delta is not None else np.exp(orig['ms_by_store_week'])
    orig['w_new'] = 0
    count = 0
    
    # Iterate contraction mapping until threshold is found
    while True:
        if any(orig['w_old'].isnull()):
            print(count)
            raise Exception("NaNs")

        orig = sim_shares(orig,sigma_B,sigma_I)
        orig['w_new'] = orig['w_old']*orig['ms_by_store_week']/orig['est']
        
        if np.average(np.log(orig['ms_by_store_week']/orig['est']).abs()) < epsilon:
            break
        if count > 100:
            print("over_count")
            print(np.average(np.log(orig['ms_by_store_week']/orig['est'])))
            break
        orig['w_old'] = orig['w_new']
        count += 1

    return np.log(orig['w_new']).to_numpy()

# Calculate xi: our jt-level residual
# Two parts: iterate contraction mapping, then subtract out linear terms given beta
def calc_xi(data,beta): 

    # Calculate xi: take linear component out of delta
    data['xi'] = data['delta'] - data[['price_', 'brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].dot(beta)
    return data['xi'].to_numpy()

Below I try to run the above code step by step. Keeping the mess so that you can see the (lack of) convergence: the outputted numbers are the difference between successive values of $\delta_{jt}$, averaged over all jt-pairs. It looks like there's initial convergence, but eventually it starts to explode...

## Step 3: Define GMM objective function

We now use our instruments to define an objective function which is to be minimized to find our optimal paramters $\beta$, $\sigma_{B}$, and $\sigma_{I}$. Using the formula found in Nevo's RA guide, we can express $\beta$ as a function of $(\sigma_{B},\sigma_{I})$: 
$$\beta = (X^{T}ZWZ^{T}X)^{-1}X^{T}ZWZ^{T}\delta(\sigma_{B},\sigma_{I})$$  
With $\beta$ in hand, we can now calculate $\xi(\sigma_{B},\sigma_{I},\beta)$ and thus our entire objective function:
$$\xi^{T}ZWZ^{T}\xi$$

In [ ]:
master = data.copy()

#kxn
Z = instr.to_numpy()

# nxn
W = np.linalg.inv(np.matmul(np.transpose(Z),Z))

# 12xn
X = master[['price_', 'brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].to_numpy()

#12x12
proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))

def gmm_obj(sigma, data):
    sigma_B = sigma[0]
    sigma_I = sigma[1]

    if 'delta' in data.columns:
        delta=data['delta']
    else:
        delta = None
    data['delta'] = calc_delta(data,sigma_B,sigma_I,delta)
    #1x1
    vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),data['delta']))))
    beta = np.matmul(proj,vect)
    # 1Xn
    xi = calc_xi(data,beta)

    # 1x1 
    ans = np.matmul(np.transpose(xi),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),xi))))
    return ans

~## Step 4: Nonlinear search over parameters~

~Now that we've defined a loss function to minimize, we look for parameters $\sigma_{B}, \sigma_{I}$ that minimize it. We use scipy's fsolve, which relies on MINPACK's hybrid algorithm, for nonlinear optimization.~

## Step 4: Linear Search over parameters

Scipy's non-linear minimize model does not funtion well when there is systematic variation. Switch, for the sake of finding $\sigma_{B}, \sigma_{I}$ to a linear search model.

In [ ]:
pd.set_option('mode.chained_assignment', None)
def search_grid(data, range_b, range_i, scale_b, scale_i):
    mses = {}
    best = 100000
    sigma_is = np.arange(range_i[0], range_i[1], scale_i)
    sigma_bs = np.arange(range_b[0], range_b[1], scale_b)
    for sigma_i in sigma_is:
        for sigma_b in sigma_bs:
            try:
                mse = gmm_obj([sigma_b, sigma_i], data)
            except Exception as e:
                print(f"Exception {e}")
                continue
            if best > mse:
                best = mse
                print(best)
                best_coef = (sigma_b, sigma_i)
            mses[(sigma_b, sigma_i)] = mse
    return (best, best_coef, mses)

In [ ]:
(best, best_coef, mses) = search_grid(data, (0, 2), (0,2), 0.5,0.5)
(best, best_coef)

In [ ]:
[sigma_B, sigma_I] = best_coef
range_B = (sigma_B -0.5, sigma_B + 0.5)
range_I = (sigma_I -0.5, sigma_I + 0.5)
(best, best_coef, mses) = search_grid(data, range_B,range_I, 0.1,  0.1)
(best, best_coef)

In [ ]:
[sigma_B, sigma_I] = best_coef
range_B = (sigma_B -0.1, sigma_B + 0.1)
range_I = (sigma_I -0.1, sigma_I + 0.1)
(best, best_coef, mses) = search_grid(data, range_B,range_I, 0.01,  0.01)
(best, best_coef)

In [ ]:
[sigma_B, sigma_I] = best_coef
range_B = (sigma_B -0.01, sigma_B + 0.01)
range_I = (sigma_I -0.01, sigma_I + 0.01)
(best, best_coef, mses) = search_grid(data, range_B,range_I, 0.001,  0.001)
(best, best_coef)

In [ ]:
[sigma_B, sigma_I] = best_coef

proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))

vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),calc_delta(data,sigma_I,sigma_B, data['delta'])))))


beta = np.matmul(proj,vect)
alpha = beta[0]
beta

## Elasticity calculation

Unlike the logit specification, elasticities under BLP need to be simulated. We will simulate: 
$$ e_{jjt} = -\frac{p_{jt}}{s_{jt}}\int (\alpha + \sigma_{I}I_{i})Pr_{ijt}(1-Pr_{ijt})dP_{D}(D)dP_{\nu}(\nu) $$ $$ e_{jkt} = \frac{p_{kt}}{s_{jt}} \int (\alpha + \sigma_{I}I_{i})Pr_{ijt}Pr_{ikt}dP_{D}(D)dP_{\nu}(\nu)$$
where $Pr_{ijt}$ is the probability of $i$ choosing $j$, simulated using the procedure in step 1.

In [ ]:
for i in range(1,12):
    data.loc[data['brand'] == i, f'price_{i}'] = data.loc[data['brand'] == i, 'price_']
    data.loc[data['brand'] != i, f'price_{i}'] = 0

In [ ]:
R = 100
def calc_own_e(orig,alpha,sigma_B,sigma_I):

    # e_own is total of all simulated share-price derivatives
    orig['running_e_own'] = 0

    # Simulate draws
    for i in range(R):
        data_copy = orig.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)
    
        data_copy['dsdp'] = (alpha + sigma_I*data_copy['hhincome'+str(hh)])*data_copy['ms_by_store_week']*(1-data_copy['ms_by_store_week'])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig['running_e_own'] = orig['running_e_own'] + orig['dsdp']
        orig = orig.drop(columns=['dsdp'])

    # Calculate elasticity
    orig['e_own'] = orig['price_']*orig['running_e_own']/(orig['ms_by_store_week']*R)
    return orig


def calc_cross_e(orig,alpha,sigma_I,k):
    orig[f'e_{k}'] = 0
    
    for i in range(R):
        data_copy = orig.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)

        # Calculate utility
        data_copy['V'] = data_copy['w_old']*np.exp(sigma_B*nu + sigma_I*data_copy[f'hhincome{hh}']*data_copy['price_'])

        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['s'] = data_copy['V']/data_copy['sum']

        data_k = orig[orig['brand']==k]
        data_k = data_k.rename(columns={'ms_by_store_week':'s_k'})
        data_k = data_k[['store','week','brand','s_k']]
        data_copy = pd.merge(data_copy,data_k.drop(columns=['brand']),how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['dsdp'] = data_copy['s']*(data_copy['s_k'])*(alpha + sigma_I*data_copy['hhincome'+str(hh)])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig[f'e_{k}'] = orig[f'e_{k}'] + orig['dsdp']
        print(orig[f'e_{k}'])
        orig = orig.drop(columns=['dsdp'])
    
    orig[f'e_{k}'] = orig[f'e_{k}']*orig[f'price_{k}']/(orig['ms_by_store_week']/R)
    
    return orig

In [ ]:
# 2b: Elasticities for store 9, week 10
alpha = beta[0]

data = calc_own_e(data,alpha,sigma_B,sigma_I)
for i in range(1,12):
    data = calc_cross_e(data,alpha,sigma_I,i)

data_ans = data_ans[['brand','e_own','e_1','e_2','e_3','e_4','e_5','e_6','e_7','e_8','e_9','e_10','e_11']]
for i in range(1,12):
    data.loc[data['brand']==i, f'e_{i}'] = data.loc[data['brand']==i, 'e_own']

In [ ]:
price_elasticities = data.groupby('brand')[[f'e_{i}' for i in range(1,12)]].aggregate('mean')
price_elasticities
